# TF-IDF of essays 
This jupyter notebook takes in a pool of essays, and constructs the TF-IDF score and trains a neural network to take in a TF-IDF matrix and compute a vocabulary score based on this information

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
from sklearn.feature_extraction.text import TfidfVectorizer
'''
this is the way to get kaggle files I think
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

#hello
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nthis is the way to get kaggle files I think\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [2]:
#read in csv file
full_train_data = pd.read_csv('./data/train.csv', index_col=0)

In [96]:
#Reads in 333,000 most common words in English language, not currently used later in code but might be useful...
all_vocab_words = pd.read_csv('./data/unigram_freq.csv', index_col=0)
all_vocabs = np.array([data[0] for data in all_vocab_words.iterrows()])

In [73]:
#parse over every row, store essay in the essays_train/test lists, and store the 6 scores in the scores_train/test list
essays_train = []
scores_train = []
essays_test = []
scores_test = []
all_essays = []
all_vocabs_words = []

#Technique for splitting essays into training and testing data and results
'''
for ind, data in enumerate(full_train_data.iterrows()):
    text, cohes, syntax, vocab, phrase, gram, convs = data[1]
    essays_train.append(text) if ind % 2 == 0 else essays_test.append(text)
    scores_train.append([cohes, syntax, vocab, phrase, gram, convs]) if ind % 2 == 0 else scores_test.append(
        [cohes, syntax, vocab, phrase, gram, convs]) '''

for ind, data in enumerate(full_train_data.iterrows()):
    text, cohes, syntax, vocab, phrase, gram, convs = data[1]
    all_essays.append(text)
    all_vocabs_words.append(vocab) 


In [74]:
#make into np arrays, currently not used
np_essays_train = np.array(essays_train)
np_essays_test = np.array(essays_test)
np_scores_train = np.array(scores_train)
np_scores_test = np.array(scores_test)

In [79]:
#make tfIdf for all essays in bank
#np_all_essays = np.concatenate((essays_train, essays_test), axis = 0)
np_all_essays = np.array(all_essays)
np_all_scores = np.array(all_vocabs_words)

In [10]:
#Implement tf-idf over all essays
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(np_all_essays)

In [97]:
#split tf-idf vectors and associated vocab scores into a training and testing 
#data set
embedded_training = []
embedded_testing = []
score_training = []
score_testing = []

for index, essay in enumerate(np_all_essays):
  tf_idf_per_word = tfIdf[index].toarray()[0]
  essay_score = np_all_scores[index]
  if index % 2 == 0:
    embedded_training.append(tf_idf_per_word)
    score_training.append(essay_score)
  else:
    embedded_testing.append(tf_idf_per_word)
    score_testing.append(essay_score)

np_embedded_training = np.array(embedded_training)
np_embedded_testing = np.array(embedded_testing)
np_score_training= np.array(score_training)
np_score_testing = np.array(score_testing)

#currently unused code

#df = pd.DataFrame(tfIdf[index].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
#tf_idf_per_word = np.array([data[1] for data in df.iterrows()])
#indices=np.array(tfIdfVectorizer.get_feature_names())
#df = df.sort_values('TF-IDF', ascending=False

In [98]:
#create word to index and index to word mapping to make looking up values easy
indices=np.array(tfIdfVectorizer.get_feature_names())
ind_to_word = {} 
word_to_ind = {}
for index, word in enumerate(indices):
  ind_to_word[index] = word
  word_to_ind[word] = index

In [45]:
'''df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print (df.head(50))'''

'df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])\ndf = df.sort_values(\'TF-IDF\', ascending=False)\nprint (df.head(50))'

**IMPORTANT**:
To reverse engineer TF-IDF, SKlearn does a few things different.

- TF is calculated by # of times word appears in document.
- IDF is calculated by log base e ([1 + # documents] / [1 + # documents word appears in])

Once all the words have TF-IDF scores, they are normalized by dividing by the square of all the other vectors
'''

In [106]:
#Will be useful later
idf_vector_for_every_word = tfIdfVectorizer.idf_

In [95]:
#number of distinct words, and therefore our input layer
num_vars = len(indices)

#at this point, we have a training set of TF-IDF vectors and a testing set, 
# as well as the associated scores. next step is to create a NN to connect them
#  and create weights

# after the NN is created, we will need to take an essay and convert it into a 
# TF-IDF vector over the same word bank, and compute 

- use LSTMS on word embeddings
